In [177]:
import pickle
import statistics
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
%matplotlib inline

In [178]:
with open('datalist.pkl', 'rb' ) as f:
    l = pickle.load(f)

In [179]:
with open('add_data.pkl', 'rb' ) as f:
    add = pickle.load(f)

In [180]:
data =  l + add

In [181]:
df = pd.DataFrame(data)

In [182]:
df.columns

Index(['Baths', 'Beds', 'Community', 'County', 'Favorite', 'Favorites',
       'Lot Size', 'Price_sqft', 'Redfin Tour', 'Redfin Tours',
       'Redfin_estimate', 'Sq. Ft.', 'Stories', 'Style', 'View', 'Views',
       'X-Out', 'X-Outs', 'Year Built', 'Year Renovated', 'all-time-Favorite',
       'all-time-Favorites', 'all-time-Redfin Tour', 'all-time-Redfin Tours',
       'all-time-X-Out', 'all-time-X-Outs', 'listed_date', 'listed_price',
       'school_ratings', 'sold_date', 'sold_price'],
      dtype='object')

In [138]:
#convert school ratings to an average
def get_avg_school_rating(row):
    avg_rating = statistics.mean(list(map(int,row['school_ratings'] )))
    return avg_rating

In [139]:
df['avg_school_rating'] = df.apply(get_avg_school_rating, axis = 1)

In [140]:
df = df.drop('school_ratings',1)

In [142]:
df = df.drop_duplicates()

In [143]:
df['Favorites'] = df['Favorites'].fillna(value = 1)
df = df.drop('Favorite', 1)

In [144]:
df['Redfin Tours'] = df['Redfin Tours'].fillna(value = 1)
df = df.drop('Redfin Tour', 1)

In [145]:
df['Views'] = df['Views'].fillna(value = 1)
df = df.drop('View', 1)

In [146]:
df['X-Outs'] = df['X-Outs'].fillna(value = 1)
df = df.drop('X-Out', 1)

In [147]:
houses = df[df['listed_price'].isnull() == False]

In [148]:
houses = houses[['Baths', 'Beds', 'Community', 'County', 'Favorites', 'Lot Size','Price_sqft', 'Redfin Tours', 
                 'Redfin_estimate', 'Sq. Ft.', 'Stories',
                 'Style', 'Views', 'X-Outs', 'Year Built', 'Year Renovated','listed_date', 
                 'listed_price', 'sold_date', 'sold_price','avg_school_rating']]

In [149]:
houses = houses[houses.Beds != '—']
houses = houses[houses.Baths != '—']
houses = houses.drop("County", 1)
houses = houses.drop('Lot Size', axis = 1)
houses = houses.drop('Stories', axis = 1)

In [150]:
def clean_sold_price(row):
    price = row['sold_price'].split()
    return price[0]

houses['cleaned_sold_price'] = houses.apply(clean_sold_price, axis = 1)
houses = houses.drop('sold_price',1)

In [151]:
houses.columns = ['Baths', 'Beds', 'Community', 'Favorites', 'Price_sqft', 'Redfin_Tours',
       'Redfin_estimate', 'SqFt.', 'Style', 'Views', 'X-Outs', 'Year_Built',
       'Year_Renovated', 'listed_date', 'listed_price', 'sold_date',
       'avg_school_rating', 'cleaned_sold_price']

In [152]:
houses['Baths'] = pd.to_numeric(houses['Baths'])
houses['Beds'] = pd.to_numeric(houses['Beds'])
houses['Favorites'] = pd.to_numeric(houses['Favorites'])
houses['Redfin_Tours'] = pd.to_numeric(houses['Redfin_Tours'])
houses['Views'] = houses['Views'].str.replace(',' , "")
houses['Views'] = pd.to_numeric(houses['Views'])
houses['X-Outs'] = pd.to_numeric(houses['X-Outs'])
#houses = houses2[houses2.Redfin_estimate != 'NA']
houses['Year_Built'] = houses['Year_Built'].astype(int)

In [153]:
def get_numeric_price(row):
    price = row['cleaned_sold_price'].replace(',', "")
    price = price.replace("$", "")
    return price
houses['sold_price'] = houses.apply(get_numeric_price, axis=1)
houses['sold_price'] = pd.to_numeric(houses['sold_price'])
houses = houses.drop('cleaned_sold_price', 1)

In [154]:
def get_numeric_price(row):
    price = row['Listed_Price'].replace(',', "")
    price = price.replace("$", "")
    return price
houses.columns = ['Baths', 'Beds', 'Community', 'Favorites', 'Price_sqft', 'Redfin_Tours',
       'Redfin_estimate', 'SqFt.', 'Style', 'Views', 'X-Outs', 'Year_Built',
       'Year_Renovated', 'listed_date', 'Listed_Price', 'sold_date',
       'avg_school_rating', 'sold_price']
houses = houses[houses.Listed_Price != '* ']
houses['Listed_Price'] = houses.apply(get_numeric_price, axis=1)
houses['Listed_Price'] = pd.to_numeric(houses['Listed_Price'])

In [155]:
def get_numeric_sqft(row):
    price = row['SqFt.'].replace(',', "")
    price = price.replace("$", "")
    return price
houses['SqFt'] = houses.apply(get_numeric_sqft, axis=1)
houses['SqFt'] = pd.to_numeric(houses['SqFt'])
houses = houses.drop('SqFt.', 1)

In [156]:
houses['Price_sqFt'] = houses['Listed_Price']/houses['SqFt']

In [157]:
#create age from Year Built
houses['age'] = 2021 - houses['Year_Built'] 

#create variables from Year Renovated
houses['Renovated'] = np.where( houses['Year_Renovated'] == '—', 0, 1)

In [158]:
#create variables from year renovated
def recently_renovated(row):
    if row['Year_Renovated'] == '—':
        return 0
    elif int(row['Year_Renovated']) > 2010:
        return 1
    else:
        return 0

houses['Recently_Renovated'] = houses.apply(recently_renovated, axis=1)

In [159]:
#transform listed date and sold date to dates format 
def get_date(row):
    d = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, "Jul": 7, "Aug": 8, "Sep":9, "Oct":10, "Nov": 11, "Dec": 12}
    month = d[row['sold_date'].split()[0]]  
    day = int(row['sold_date'].split()[1][:-1])
    year = int(row['sold_date'].split()[2])
    return datetime.datetime(year, month, day)
houses['date_sold'] = houses.apply(get_date, axis =1)
houses = houses.drop('sold_date', 1)

In [160]:
#transform listed date and sold date to dates format 
def get_date(row):
    d = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, "Jul": 7, "Aug": 8, "Sep":9, "Oct":10, "Nov": 11, "Dec": 12}
    month = d[row['listed_date'].split()[0]]  
    day = int(row['listed_date'].split()[1][:-1])
    year = int(row['listed_date'].split()[2])
    return datetime.datetime(year, month, day)
houses['date_listed'] = houses.apply(get_date, axis =1)
houses = houses.drop('listed_date', 1)

In [161]:
#create variable days on the market
houses['days_on_market'] = houses['date_sold'] - houses['date_listed']
houses['days_on_market'] = houses['days_on_market'].dt.days

In [162]:
#create neighborhood groups,
d = pd.DataFrame(add)
neighborhoods = list(d.Community)
bellevue = neighborhoods[:315]
kirkland = neighborhoods[315:652]
seattle = neighborhoods[652:]
d = pd.DataFrame(l)
s = list(d.Community)
seattle += s
def get_community(row, bellevue, kirkland):
    community = {'North': ['Maple Leaf',  'Greenwood', 'Bitter Lake', 'View Ridge',  'Ravenna','Northgate','Shoreline',  'North Seattle',
                       'Broadview', 'Bryant','Cedar Park', 'Crown Hill', 'Haller Lake','Hawthorne Hills',
                     'Inverness', 'Lake City','Lake Forest Park','Laurelhurst', 'Licton Springs', 'Meadowbrook',
                     'North Beach','North College Park','Olympic Hills','Parkwood','Pinehurst','Roosevelt','Sand Point',
                    'Victory Heights', 'View Ridge','Wedgwood', 'Windermere'],
           'South': ['Mt Baker','White Center',  'Beacon Hill', 'Top Hat','Columbia City', 'Brighton','Rainier Beach', 'Judkins',
          'Seward Park', 'Rainier Valley', 'Arbor Heights','Arroyo', 'Boulevard Park','Brighton',  'Delridge', 'Dunlap', 
          'Fauntleroy',  'Gatewood','Genesee','Glen Acres', 'High Point','Highland Park','Hillman City', 'Lakeridge','Lakewood',
          'Lincoln Park','Matthews Beach','Marine View','Mt Baker',  'Puget Ridge','Rainier Beach','Rainier Valley', 'Riverton',
          'Seward Park','South Park','Top Hat', 'Upper Rainier Beach','Westwood','Westwood Village', 'Seattle'],
           'Central Area': ['Leschi', 'First Hill','Capitol Hill', 'Central Area','Madrona','Madison Valley', 'Broadway', 
                 'Madison Park','Montlake', 'Queen Anne', 'Eastlake','Westlake','Queen Anne'],
           'Ballard': ['Ballard',   'Loyal Heights', 'Sunset Hill','West Woodland', 'Whittier', 'Magnolia'],
           'West Seattle': ['Alaska Junction', 'West Seattle','Alki', 'Belvidere', 'Admiral','Fairmount', 'Junction', 'North Admiral','Seaview'],
           'Downtown':  ['Downtown', 'Lake Union','South Lake Union','Denny Triangle', 'Belltown', 'Downtown Seattle',  'Cascade',  'Denny Regrade', 'International District'],
           'Fremont' : [ 'Wallingford', 'Fremont',  'Green Lake','Phinney Ridge', 'Northlake',  'University District', 'Woodland Park','Magnolia-Queen Anne'],
                'Bellevue': bellevue, 'Kirkland': kirkland }
    for key, value in community.items():
        if row['Community'] in value:
            return key 
        
houses['neighborhood'] = houses.apply(get_community, args= (bellevue, kirkland), axis=1)       

In [163]:
#create style groups
houses['Style'] = houses['Style'].map({'Condo/Co-op': 'Condo/Co-op', 'Single Family Residential': 'Single Family Residential',
                                       'Multi-Family (2-4 Unit)': 'Multi-Family','Townhouse': 'Townhouse', 'Other': 'Multi-Family', 
                                       'Multi-Family (5+ Unit)': 'Multi-Family' })

In [164]:
houses['Views'] = houses['Views'].fillna(value = 1)

In [165]:
#get numeric redfin estimate
houses = houses[houses.Redfin_estimate != 'NA']
def get_numeric_price(row):
    price = row['Redfin_estimate'].replace(',', "")
    price = price.replace("$", "")
    return price

houses['Redfin_estimate'] = houses.apply(get_numeric_price, axis=1)
houses['Redfin_estimate'] = pd.to_numeric(houses['Redfin_estimate'])

In [166]:
#create variable of interest
houses['over_under_ask'] = (houses['sold_price'] - houses['Listed_Price'])/houses['Listed_Price']

In [167]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1315 entries, 0 to 2529
Data columns (total 25 columns):
Baths                 1315 non-null float64
Beds                  1315 non-null int64
Community             1315 non-null object
Favorites             1315 non-null int64
Price_sqft            1315 non-null object
Redfin_Tours          1315 non-null int64
Redfin_estimate       1315 non-null int64
Style                 1315 non-null object
Views                 1315 non-null float64
X-Outs                1315 non-null int64
Year_Built            1315 non-null int32
Year_Renovated        1315 non-null object
Listed_Price          1315 non-null int64
avg_school_rating     1315 non-null float64
sold_price            1315 non-null int64
SqFt                  1315 non-null int64
Price_sqFt            1315 non-null float64
age                   1315 non-null int32
Renovated             1315 non-null int32
Recently_Renovated    1315 non-null int64
date_sold             1315 non-null datet

In [168]:
#save cleaned data ready for analysis
with open('cleaned_data.pkl', 'wb' ) as f:
    pickle.dump(houses, f)